In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0
torch:  2.3 ; cuda:  2.3.1
detectron2: 0.6


In [2]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [3]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, r"E:\Random Python Scripts\FarmLand-Segmentation-main\Detectron 2\Cleaned Dataset (Augmented, COCO, 20k)\train\_annotations.coco.json", r"E:\Random Python Scripts\FarmLand-Segmentation-main\Detectron 2\Cleaned Dataset (Augmented, COCO, 20k)\train")
register_coco_instances("my_dataset_val", {}, r"E:\Random Python Scripts\FarmLand-Segmentation-main\Detectron 2\Cleaned Dataset (Augmented, COCO, 20k)\valid\_annotations.coco.json", r"E:\Random Python Scripts\FarmLand-Segmentation-main\Detectron 2\Cleaned Dataset (Augmented, COCO, 20k)\valid")

In [4]:
train_metadata = MetadataCatalog.get("my_dataset_train")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train")

[07/10 11:44:36 d2.data.datasets.coco]: Loading E:\Random Python Scripts\FarmLand-Segmentation-main\Detectron 2\Cleaned Dataset (Augmented, COCO, 20k)\train\_annotations.coco.json takes 1.88 seconds.
WARNING [07/10 11:44:36 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[07/10 11:44:37 d2.data.datasets.coco]: Loaded 18407 images in COCO format from E:\Random Python Scripts\FarmLand-Segmentation-main\Detectron 2\Cleaned Dataset (Augmented, COCO, 20k)\train\_annotations.coco.json


In [5]:
val_metadata = MetadataCatalog.get("my_dataset_val")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val")

WARNING [07/10 11:44:38 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[07/10 11:44:38 d2.data.datasets.coco]: Loaded 749 images in COCO format from E:\Random Python Scripts\FarmLand-Segmentation-main\Detectron 2\Cleaned Dataset (Augmented, COCO, 20k)\valid\_annotations.coco.json


In [6]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.OUTPUT_DIR = "E:\Random Python Scripts\FarmLand-Segmentation-main\Detectron 2\model_50000_epochs (7.5k)"
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

[07/10 11:44:38 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from E:\Random Python Scripts\FarmLand-Segmentation-main\Detectron 2\model_50000_epochs (7.5k)\model_final.pth ...


In [7]:
dir_path = r"E:\Random Python Scripts\FarmLand-Segmentation-main\Detectron 2\Cleaned Dataset (Augmented, COCO, 20k)\test"

jpg_files = [f for f in os.listdir(dir_path) if f.endswith(".jpg")]

random_file = random.choice(jpg_files)

file_path = os.path.join(dir_path, random_file)

new_im = cv2.imread(file_path)

outputs = predictor(new_im)

v = Visualizer(new_im[:, :, ::-1], metadata=train_metadata, instance_mode=ColorMode.SEGMENTATION)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

cv2.imshow("Segmented Image", out.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\Siddharth\anaconda3\envs\detectron_env\lib\site-packages\torch\functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3588.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
